In [1]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import numpy as np
import pandas as pd

In [2]:
Chung_nam = pd.read_csv('mini_project_1/Jeollabuk.csv')
Chung_nam

,Unnamed: 0,업종,업소명,주요품목,가격,연락처,주소
0,0,한식_일반,명지해물찜칼국수,바지락탕,10000원,041-662-3938,충청남도 서산시 대산읍 물안2길 6
1,1,한식_면류,갈산식당,칼국수,7000원,041-632-1209,"충청남도 홍성군 갈산면 갈산로 115, 105호"
2,2,이미용업,세븐데이남성커트,커트,9000원,041-568-6808,"충청남도 천안시 서북구 두정로 212, 107호 (두정동)"
3,3,한식_분식,윤경이네분식,김밥,2000원,041-555-8898,충청남도 천안시 동남구 버들1길 29(원성동)
4,4,한식_일반,우리집한식뷔페,한식뷔페,7000원,010-8222-4952,충청남도 천안시 서북구 입장면 연곡길 166
...,...,...,...,...,...,...,...
320,320,세탁업,선경세탁소,신사복드라이,7000원,041-574-7001,충청남도 천안시 동남구 양지4길 15(봉명동)
321,321,이미용업,챠밍헤어클럽미용실,커트,10000원,041-569-6005,충청남도 천안시 동남구 고재4길 55(원성동)
322,322,이미용업,스타미용실,커트,5000원,041-562-0155,충청남도 천안시 동남구 대흥로 271(대흥동)
323,323,이미용업,포인트미용실,커트,3000원,041-567-8301,"충청남도 천안시 동남구 공설시장2길 3, 3층(대흥동)"


In [3]:
Chung_nam.drop('Unnamed: 0',axis = 1, inplace = True)

Chung_nam['주소_구,군'] = Chung_nam['주소'].apply(lambda x:x.split(' ')[0])

Chung_nam['주소_구,군'] = Chung_nam['주소'].apply(lambda x:x.replace('\xa0',' '))
Chung_nam['keyword'] = Chung_nam['주소_구,군']

Chung_nam_address_list = Chung_nam['keyword'].apply(lambda x:x.split(' '))

new_Chung_nam_address_list = []

for i,item in enumerate(Chung_nam_address_list):
    #print(ad_li[i][:2])
    address = item[0] + ' ' + item[1]
    new_Chung_nam_address_list.append(address)
    
new_Chung_nam_address_list

Chung_nam['keyword'] = new_Chung_nam_address_list

Chung_nam['keyword'] = Chung_nam['keyword'] + ' ' + Chung_nam['업소명']

In [5]:
# 카카오맵 url 가져오기
url = 'https://map.kakao.com/'

# 크롬 드라이버 실행하기
driver = webdriver.Chrome()

# 카카오맵 url로 이동하기
driver.get(url)

score_list = []

for key in Chung_nam['keyword'][:325]:
    # 검색창 클릭하기
    search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

    # keyword의 key를 검색창에 넣기
    search.send_keys(key)

    # 엔터키로 이동하기
    search.send_keys(Keys.RETURN)

    time.sleep(2)

    # 웹 스크립트 받아오기
    html = driver.page_source

    # 웹스크립트에서 원하는 콘텐츠 추출하기 - parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    # keyword별 평점 가져오기
    try:
        # 평점 path 가져오기
        score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
        
        # score_path를 split한 후 3번째 문자 추출하기
        score = soup.select(score_path)[0].text.split('\n')[3]
        
        # 평점을 빈 리스트 L에 추가하기
        score_list.append(score)
        
    except:
        score_list.append('별점 없음')
        
    search.clear()
    
score_list

Chung_nam['평점'] = score_list
Chung_nam.drop('주소_구,군',axis = 1, inplace = True)
Chung_nam.drop('keyword',axis = 1, inplace = True)
Chung_nam.to_csv('Chung_namscore.csv', encoding='utf-8-sig')

<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:33: SyntaxWarning: invalid escape sequence '\.'
C:\Users\vvv\AppData\Local\Temp\ipykernel_18292\1409708911.py:14: SyntaxWarning: invalid escape sequence '\.'
  search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')
C:\Users\vvv\AppData\Local\Temp\ipykernel_18292\1409708911.py:33: SyntaxWarning: invalid escape sequence '\.'
  score_path = '#info\.search\.place\.list > li:nth-child(1) > div.rating.clickArea'
